In [3]:
import mysql.connector
import tkinter as tk
from tkinter import ttk
from io import BytesIO
from PIL import Image, ImageTk
import pygame
import requests

# Connect to the voice_club database
cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="salah",
    database="voice_club"
)

# Define the queries
queries = [
    ("List all people", "SELECT first_name, last_name, email FROM person;"),
    ("List all students and their majors", "SELECT first_name, last_name, major FROM student JOIN person ON student.student_id = person.person_id;"),
    ("List all faculty and their departments", "SELECT first_name, last_name, name AS department FROM faculty JOIN department ON faculty.department = department.department_id JOIN person ON faculty.faculty_id = person.person_id;"),
    ("List all clubs", "SELECT name, description FROM club;"),
    ("List all club advisors", "SELECT club.name, person.first_name, person.last_name FROM club JOIN faculty ON club.faculty_id = faculty.faculty_id JOIN person ON faculty.faculty_id = person.person_id;"),
    ("List all songs and their genres", "SELECT title, name AS genre FROM song JOIN genre ON song.genre_id = genre.genre_id;"),
    ("List all events and their locations", "SELECT event_id, description, name AS location FROM event JOIN location ON event.location_id = location.location_id;"),
    ("List all performances and their events", "SELECT performance_id, event.event_id, event.description FROM performance JOIN event ON performance.event_id = event.event_id;"),
    ("List all students and their majors in each club", "SELECT person.first_name, person.last_name, student.major FROM person JOIN student ON person.person_id = student.student_id JOIN membership ON student.student_id = membership.student_id JOIN club ON membership.faculty_id = club.faculty_id;"),
    ("List the number of members in each club", "SELECT club.name, COUNT(DISTINCT membership.student_id) AS num_members FROM club JOIN membership ON club.faculty_id = membership.faculty_id GROUP BY club.club_id;"),
    ("List all students who have attended at least one event", "SELECT person.first_name, person.last_name, COUNT(DISTINCT attendance.event_id) AS num_events FROM person JOIN student ON person.person_id = student.student_id JOIN attendance ON student.student_id = attendance.student_id GROUP BY person.person_id HAVING COUNT(DISTINCT attendance.event_id) > 0;")
]

# Define a function to execute a query and return the results as a list of strings
def execute_query(query):
    cursor = cnx.cursor()
    cursor.execute(query)
    results = cursor.fetchall()
    cursor.close()
    return [", ".join(map(str, row)) for row in results]

# Define a function to display the results under the Combobox widget
def display_results(results):
    # Clear any existing results
    for i in result_listbox.get_children():
        result_listbox.delete(i)

    # Create a new Listbox widget to display the results
    for i, result in enumerate(results):
        result_listbox.insert("", i, text=result)

# Create a tkinter window
root = tk.Tk()
root.attributes('-fullscreen', False)
root.geometry("800x600")
root.title("Voice Club Database Queries")

# Download the wallpaper image
response = requests.get('https://th.bing.com/th/id/OIP.4nKgEOnGLhYXtoF8V6e8hgHaHa?w=183&h=183&c=7&r=0&o=5&dpr=1.3&pid=1.7')
wallpaper = Image.open(BytesIO(response.content))
wallpaper = wallpaper.resize((800, 600), Image.ANTIALIAS)
wallpaper = ImageTk.PhotoImage(wallpaper)

# Initialize pygame mixer and load the music file
pygame.mixer.init()
pygame.mixer.music.load("onlymp3.to - One Piece OP 1 - We Are! Lyrics-IHhNTt3oxtY-256k-1654143359353.mp3")

# Add a label to display the wallpaper
background_label = tk.Label(root, image=wallpaper)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Add a Combobox widget to select the query
query_combobox = ttk.Combobox(root, values=[q[0] for q in queries], state="readonly", width=60)
query_combobox.place(relx=0.5, rely=0.2, anchor="center")

# Add a Button widget to execute the selected query
execute_button = tk.Button(root, text="Execute Query", command=lambda: display_results(execute_query(queries[query_combobox.current()][1])), width=20, height=2)
execute_button.place(relx=0.5, rely=0.3, anchor="center")

# Add a Listbox widget to display the results
result_listbox = ttk.Treeview(root, columns=["Result"])
result_listbox.place(relx=0.5, rely=0.6, anchor="center")
result_listbox.heading("#0", text="Result")

# Define a Boolean variable to keep track of the music playback state
music_paused = False

# Define a function to toggle the music playback state and update the button text
def toggle_music():
    global music_paused
    if music_paused:
        pygame.mixer.music.unpause()
        music_paused = False
        music_button.configure(text="Pause Music")
    else:
        pygame.mixer.music.pause()
        music_paused = True
        music_button.configure(text="Resume Music")

# Add a button widget to the tkinter window to start and pause the music
music_button = tk.Button(root, text="Pause Music", command=toggle_music, width=20, height=2)
music_button.place(relx=0.5, rely=0.10, anchor="center")

# Start playing the background music
pygame.mixer.music.play(-1)

# Start the tkinter event loop
root.mainloop()

# Stop the background music and close the database connection
pygame.mixer.music.stop()
cnx.close()